# Vollständige First-Level Analyse

### module importieren

In [ ]:
import glob

import numpy as np
import pandas as pd
from sklearn import preprocessing

from nilearn import plotting, image, input_data
from nistats import first_level_model, reporting, regression

import seaborn as sns
import matplotlib.pyplot as plt

sns.set_context('talk')

### Strukturelles Bild

In [ ]:
mpr_img = '../data/sub-01_T1w_space-MNI152NLin2009cAsym_preproc.nii.gz'

In [ ]:
plotting.plot_anat(mpr_img);

### fMRT-Daten

In [ ]:
fmri_img_list = glob.glob('../data/sub-01_task-imagery_run-0*_bold_space-MNI152NLin2009cAsym_preproc.nii.gz')
fmri_img_list.sort()
fmri_img_list = fmri_img_list[1:]
fmri_img_list

Beispiel:

In [ ]:
mean_epi_img = image.mean_img(fmri_img_list[-1])
plotting.plot_anat(mean_epi_img);

### alle runs hintereinanderhängen

In [ ]:
fmri_imgs = image.concat_imgs(fmri_img_list)

wie viele Volumen sind in einem run?

In [ ]:
n_volumes = image.load_img(fmri_img_list[-1]).shape[-1]
n_volumes

wie viele Volumen sind in der zusammengeführten Zeitserie?

In [ ]:
fmri_imgs.shape

### Das Design

In [ ]:
onset_csvs = glob.glob('../data/sub-01_task-imagery_run-0*_events.tsv')
onset_csvs.sort()
onset_csvs = onset_csvs[1:]
onset_csvs

Beispiel (letzter run):

In [ ]:
pd.read_csv(onset_csvs[-1],sep='\t')

### die zu analysierende Bedingung auswählen (trial_type)

In [ ]:
def make_onsets(csv_file):
    df = pd.read_csv(csv_file,sep='\t')
    df.loc[:,'trial_num'] = [('0%s'%x)[-2:] for x in df.index.get_values().astype(str)]
    df.loc[:,'trial_type'] = df.loc[:,'trial_domain']
    return df

In [ ]:
onset_df = make_onsets(onset_csvs[-1])
onset_df

<font color=orange>__Modifikation:__ Statt trial_domain (die 5 Oberbedingungen), kann man auch eine andere Spalte der Onset-Tabelle nutzen um Bedingungen zu definieren</font>

### Aus den Onsets das Design erstellen

Meta-Informationen zu den runs

In [ ]:
my_tr = 3
my_duration = n_volumes*my_tr
my_duration # dauer eines runs in sekunden (volumes x tr)

Onsets mit HRF verbinden

In [ ]:
def make_design(df,my_tr=my_tr,my_duration=my_duration):
    frame_times = np.arange(0,my_duration,my_tr)
    design_matrix = first_level_model.make_first_level_design_matrix(frame_times=frame_times,
                                                                     events=df,
                                                                     drift_model=None
                                                                    )
    return design_matrix

Beispiel:

In [ ]:
sns.heatmap(make_design(onset_df));

<font color=orange>__Modifikation:__ Drifts in das Modell aufnehmen (drift_model)</font>

<font color=red>__Modifikation:__ Den Cutoff des High-Pass Filter verändern</font>

### Ein großes Design für alle 4 runs

In [ ]:
def make_big_design(onset_csvs):
    
    design_matrix = pd.DataFrame()
    for run_num,onset_csv in enumerate(onset_csvs):
        this_onset = make_onsets(onset_csv)
        this_design = make_design(this_onset)
        this_design.index = this_design.index+my_duration*run_num
        design_matrix = pd.concat([design_matrix, this_design],sort=False)
        design_matrix = design_matrix.fillna(0)
        design_matrix.columns = ['%s-run%s'%(c,run_num) for c in design_matrix.columns]

    return design_matrix

In [ ]:
design_matrix = make_big_design(onset_csvs)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(32,32))
sns.heatmap(design_matrix,ax=ax,cmap='Greys_r')
plt.show()

### Ein Modell welches Daten und Design verbindet

In [ ]:
fmri_glm = first_level_model.FirstLevelModel(t_r=3,
                                             smoothing_fwhm=8
                                            )

<font color=green>__Modifikation:__ Der Grad an räumlicher Glättung kann variiert werden</font>  
<font color=red>__Modifikation:__ Die Maske welche die zu analysierenden Voxel definiert kann aus ../data/ gewählt und hier angewendet werden</font>

In [ ]:
fmri_glm = fmri_glm.fit(fmri_imgs,design_matrices=design_matrix)

Welche Einstellungen hat das Modell?

In [ ]:
fmri_glm

Wie sieht der zu analysierende Bereich aus?

In [ ]:
display = plotting.plot_anat(mean_epi_img)
display.add_contours(fmri_glm.masker_.mask_img_)

In [ ]:
display = plotting.plot_anat(mean_epi_img,display_mode='x')
display.add_contours(fmri_glm.masker_.mask_img_)

### Wie korrelieren die Regressoren der Design-Matrix miteinander?

In [ ]:
design_matrix = fmri_glm.design_matrices_[-1]

In [ ]:
corr_df = design_matrix.corr()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(32,32))
sns.heatmap(corr_df,
            cmap='RdBu_r',
            square=True,
            vmin=-1,
            vmax=1,
            ax=ax)
plt.show()

## Ergebnisse berechnen

Wie viele runs haben wir?

In [ ]:
n_runs = len(fmri_img_list)
n_runs

###  Die Bedingungen definieren

Jeder der Bedingungen wird mit 1en an den Stellen an denen sie in der Design-Matrix vorkommt definiert

In [ ]:
conditions = {
    'emotion':   np.array([1., 0., 0., 0., 0., 0.]*n_runs),
    'gesichter': np.array([0., 1., 0., 0., 0., 0.]*n_runs),
}

In [ ]:
def show_conds(conditions):
    con_df = pd.DataFrame(conditions).T
    plt.figure(figsize=(16,4))
    sns.heatmap(con_df,
                square=True,cbar=False,cmap='Greys_r',annot=True);
    plt.show()

In [ ]:
show_conds(conditions)

<font color=green>__Modifikation:__ Die fehlenden Bedingungen hinzufügen</font>

### Einen Kontrast definieren

In [ ]:
contrast = conditions['emotion'] - conditions['gesichter']

<font color=green>__Modifikation:__ Andere Vergleiche von zwei Bedinungen rechnen</font>  
<font color=orange>__Modifikation:__ Eine Bedingung gegen alle anderen vergleichen</font>  
<font color=red>__Modifikation:__ Eine 1-faktorielle ANOVA mit 5 Stufen formulieren</font>

In [ ]:
def show_conts(contrast,design_matrix=design_matrix):
    con_df = pd.DataFrame(contrast,index=design_matrix.columns).T
    plt.figure(figsize=(16,4))
    sns.heatmap(con_df,
                square=True,cbar=False,cmap='Greys_r',annot=True);
    plt.yticks([])
    plt.show()

In [ ]:
show_conts(contrast)

### Eine Ergebniskarte generieren

In [ ]:
result_map = fmri_glm.compute_contrast(contrast)

<font color=green>__Modifikation:__ z-Werte, t-Werte und p-Werte plotten</font>  
<font color=red>__Modifikation:__ Eine 1-faktorielle ANOVA rechnen</font>

### Die Karte visualisieren

In [ ]:
display = plotting.view_img(result_map,bg_img=mpr_img,threshold=3)

<font color=green>__Modifikation:__ Einen anderen Schwellenwert auswählen</font>  
<font color=red>__Modifikation:__ Eine Karte mit t-Werten plotten, die bei p<0.001 gethresholded ist</font>  

In [ ]:
display

<font color=green>__Modifikation:__ display in neuem Fenster im Vollbildmodus öffnen</font>  
<font color=red>__Modifikation:__ Eine Tabelle mit Aktivierungen als Koordinaten ausgeben lassen</font>